In [7]:
import pickle
import torch

In [8]:
folder="/data/home/umang/Trajectory_project/GPS-MTM/outputs/mtm_test/2025-08-16_15-56-29/test_outputs"
pred_pickle_path="{}/predictions_batch_0.pkl".format(folder)
batch_pickle_path = "{}/ground_truth_batch_0.pkl".format(folder)
attention_pickle_path = "{}/attention_masks_batch_0.pkl".format(folder)
masks_pickle_path = "{}/masks_batch_0.pkl".format(folder)
with open(pred_pickle_path, 'rb') as f:
    predictions = pickle.load(f)

with open(batch_pickle_path, 'rb') as f:
    batch = pickle.load(f)

with open(attention_pickle_path, 'rb') as f:
    attention_masks = pickle.load(f)

with open(masks_pickle_path, 'rb') as f:
    masks = pickle.load(f)

In [9]:
len(masks["states"])

221

In [10]:
predictions["states"].shape

torch.Size([32, 221, 28])

In [11]:
predictions["actions"].shape

torch.Size([32, 221, 11])

In [12]:
# get the most frequently top-5 most repeated in ground_truth_list_States
# #don't use torch top-k..
ground_truth_list_states = []
for i in range(len(attention_masks)):
     # first zero mask is first time when attention_masks[i] becomes zero
    zero_indices = (attention_masks[i].flatten() == 0).nonzero(as_tuple=True)[0]
    first_zero_mask = zero_indices[0].item() if len(zero_indices) > 0 else attention_masks[i].numel()
    
    ground_truth_states= batch["states"][i, :first_zero_mask, :]
    ground_truth_states = torch.argmax(ground_truth_states, dim=-1)
    ground_truth_list_states.append(ground_truth_states)

k=5
ground_truth_list_states = torch.cat(ground_truth_list_states).flatten()
unique, counts = torch.unique(ground_truth_list_states, return_counts=True)
top_k_ground_truth = unique[torch.topk(counts, k=k).indices]
top_k_ground_truth

top_1_ground_truth = unique[torch.topk(counts, k=1).indices]
top_5_ground_truth = unique[torch.topk(counts, k=5).indices]
top_10_ground_truth = unique[torch.topk(counts, k=10).indices]
top_20_ground_truth = unique[torch.topk(counts, k=20).indices]

In [13]:
predictions_list_states=[]
predictions_list_actions=[]
ground_truth_list_states=[]
ground_truth_list_actions=[]
correct_list=[]

for i in range(len(attention_masks)):
    # first zero mask is first time when attention_masks[i] becomes zero
    zero_indices = (attention_masks[i].flatten() == 0).nonzero(as_tuple=True)[0]
    first_zero_mask = zero_indices[0].item() if len(zero_indices) > 0 else attention_masks[i].numel()
    
    print("first_zero_mask: ", first_zero_mask)
    predictions_states= predictions["states"][i, :first_zero_mask, :]
    predictions_actions= predictions["actions"][i, :first_zero_mask, :]
    ground_truth_states= batch["states"][i, :first_zero_mask, :]
    ground_truth_actions= batch["actions"][i, :first_zero_mask, :]

    # take argmax of predictions states on last dimension
    predictions_states = torch.argmax(predictions_states, dim=-1)
    ground_truth_states = torch.argmax(ground_truth_states, dim=-1)

    predictions_list_states.append(predictions_states)
    predictions_list_actions.append(predictions_actions)
    ground_truth_list_states.append(ground_truth_states)
    ground_truth_list_actions.append(ground_truth_actions)

    print(ground_truth_states)
    correct = (predictions_states == ground_truth_states).float()
    correct_list.append(correct)


first_zero_mask:  65
tensor([22,  0, 22,  0, 22,  0, 22,  0, 22,  0, 22, 16, 22,  0, 22,  0, 22,  0,
        22,  0, 22,  0, 22,  0, 22,  0, 22,  0, 22, 16, 22,  0, 22,  0, 22,  0,
        22,  0, 22,  0, 22,  0, 22,  0, 22,  8, 22,  0, 22,  0,  4, 22,  0, 22,
         0, 22,  0, 22,  0, 22,  0, 22,  0,  3,  0])
first_zero_mask:  82
tensor([22,  9, 16, 22, 10, 22,  9, 16, 22, 16, 22, 10, 22,  9, 16, 22,  9, 22,
        22, 22, 16, 22, 10, 22,  9, 16, 22,  9, 16, 22,  4, 16, 22, 16, 22, 16,
        22,  4, 22, 22, 16, 22, 10, 22,  9, 16, 22,  9, 16, 22,  4, 16, 22, 16,
        22, 16, 22,  4, 22, 22, 16, 22,  9, 22,  4, 16, 22,  9, 22, 16, 22, 10,
        22,  9, 16, 22, 10, 22, 16, 22,  4, 22])
first_zero_mask:  75
tensor([22, 16, 22,  6, 22,  9,  4, 16, 22, 16, 22, 16, 22, 26, 22, 16, 22, 18,
        22, 22, 22, 16, 22,  6, 22,  6, 22, 16, 22, 16, 22,  6, 22, 16, 22, 18,
        27, 22,  6, 22,  6, 22, 16, 22, 16, 22,  6, 22, 16, 22, 18, 27, 22,  9,
         4, 16, 22, 16, 22,  9,  4,

In [20]:
predictions_list_states=torch.cat(predictions_list_states).flatten()
ground_truth_list_states=torch.cat(ground_truth_list_states).flatten()
predictions_list_states.shape, ground_truth_list_states.shape

(torch.Size([2771]), torch.Size([2771]))

In [32]:
# get the number of correct predictions for classes in top_10_ground_truth
top_k=[top_1_ground_truth, top_5_ground_truth, top_10_ground_truth, top_20_ground_truth]
for i in range(len(top_k)):
    filtered_gt = ground_truth_list_states[(ground_truth_list_states.unsqueeze(-1) == top_k[i]).any(dim=-1)]
    filtered_pred = predictions_list_states[(ground_truth_list_states.unsqueeze(-1) == top_k[i]).any(dim=-1)]

    # get the number of correct predictions and accuracy
    correct_predictions = (filtered_pred == filtered_gt).sum()
    accuracy = correct_predictions / filtered_gt.numel()
    print(f"Number of correct predictions in {top_k[i]} ground truth: {correct_predictions.item()}")
    print(f"Accuracy of correct predictions in {top_k[i]} ground truth: {accuracy.item()}")

Number of correct predictions in tensor([22]) ground truth: 0
Accuracy of correct predictions in tensor([22]) ground truth: 0.0
Number of correct predictions in tensor([22, 16,  3, 27,  0]) ground truth: 0
Accuracy of correct predictions in tensor([22, 16,  3, 27,  0]) ground truth: 0.0
Number of correct predictions in tensor([22, 16,  3, 27,  0, 18, 20,  4, 10,  9]) ground truth: 25
Accuracy of correct predictions in tensor([22, 16,  3, 27,  0, 18, 20,  4, 10,  9]) ground truth: 0.009652509354054928
Number of correct predictions in tensor([22, 16,  3, 27,  0, 18, 20,  4, 10,  9,  6,  8, 25, 23, 26,  7,  2, 14,
        13, 12]) ground truth: 42
Accuracy of correct predictions in tensor([22, 16,  3, 27,  0, 18, 20,  4, 10,  9,  6,  8, 25, 23, 26,  7,  2, 14,
        13, 12]) ground truth: 0.015156983397901058
